
Going to predict the spreads for games starting on march 17th


In [1]:

#import sys
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#stderr = sys.stderr
#sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
def make_network(FILENAME):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    Sike!
    
    model : object
        MLP which can predict the outcome of NBA games
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    
    print(data)

    dat = []

    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    

    """At this point in the data dat is an array arranged as 
        OR  DR  TOT   A  PF  ST  TO  BL  PTS       3P%       FG%       FT% x2  (for road then home )
    so next up is to hot pull road and home points 
    
    road points is column 8 and home points is column 20. 
    
    So grab those from dat and make y. 
    
    X is every other column, so merge everyone else 
    
    """
    dat = np.array(dat)   
    roadpts = dat[:,8]

    homepts = dat[:,21]
    endspreadS = roadpts-homepts
    openingspreadS = dat[:,9]
#    print("opening spread:" + str(dat[:,9]))
    y = []
    for j in range(len(endspreadS)):
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread <0:
            y.append(np.array([0,1,0]))
        elif openspread + endspread >0:
            y.append(np.array([1,0,0]))
        else: 
            y.append(np.array([0,0,1]))



    y = np.array(y)
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    model = MLPClassifier()
    model.shuffle = True
    model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))
    return model
    

If spread of road team is + that means they are supposed to lose by that much
if the end spread is - that means the road team lost by that much


In [10]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [11]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    return game

In [12]:
teams, stats =make_prediction_data('teamstats_322.csv')

Here is every teams index value: 
0     Golden State Warriors*
1     Toronto Raptors*      
2     Houston Rockets*      
3     New Orleans Pelicans  
4     Minnesota Timberwolves
5     Denver Nuggets        
6     Cleveland Cavaliers   
7     Oklahoma City Thunder 
8     Los Angeles Clippers  
9     Charlotte Hornets     
10    Brooklyn Nets         
11    Washington Wizards    
12    Indiana Pacers        
13    Los Angeles Lakers    
14    Philadelphia 76ers    
15    Portland Trail Blazers
16    Milwaukee Bucks       
17    New York Knicks       
18    Phoenix Suns          
19    Miami Heat            
20    Orlando Magic         
21    Boston Celtics*       
22    Atlanta Hawks         
23    San Antonio Spurs     
24    Chicago Bulls         
25    Utah Jazz             
26    Detroit Pistons       
27    Dallas Mavericks      
28    Sacramento Kings      
29    Memphis Grizzlies     
30    League Average        
Name: Team, dtype: object


In [13]:

noahbets = make_network('1517_boxscores_withspreads.csv')

      OR  DR  TOT   A  PF  ST  TO  BL  PTS  SPREAD       3P%       FG%  \
0     13  29  42   17  22  6   18  6   88   9.5     0.333333  0.367816   
1     11  40  51   31  22  12  14  5   117 -9.5     0.371429  0.478723   
2     6   25  31   19  19  9   11  5   104  6.0     0.333333  0.487805   
3     5   29  34   22  18  5   12  3   113 -6.0     0.684211  0.520000   
4     21  34  55   25  19  13  13  3   129  8.0     0.500000  0.479592   
5     8   27  35   24  19  11  16  6   100 -8.0     0.212121  0.470588   
6     16  36  52   27  22  5   10  7   108  4.0     0.250000  0.484536   
7     15  30  45   20  15  5   11  4   96  -4.0     0.333333  0.382022   
8     10  39  49   26  27  8   15  8   121  5.0     0.375000  0.432692   
9     8   44  52   30  23  11  16  6   130 -5.0     0.526316  0.505376   
10    15  29  44   22  19  8   16  3   117  14.0    0.340909  0.443299   
11    12  35  47   36  20  13  19  9   122 -14.0    0.343750  0.539326   
12    8   32  40   17  24  8   13  0  

In [17]:
noahbets.predict(np.array([game1,game2,game3]))

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 0]])

On March 26th, using team stats as of 3/21

Hornets
Lakers
sixers

ATS Predictions 3/21:
--------------------
Hornets Cha

Raptors Raps

76ers Sixers

x Knicks Heat

Bulls Bulls Bulls

Clips Clips Clips

Pels Pels Pels

x Wiz Spurs







In [ ]:
10 +10 +10 + 10  -11 

In [8]:
7+2+2 +3 +6

20

In [9]:
4+2+6+3+2

17

In [10]:
20/38

0.5263157894736842